In [1]:
import os, sys
sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src.tools import unfreeze, freeze
from src.tools import load_dataset, get_loader_stats

from copy import deepcopy
import json

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

/home/user/conda/lib/python3.7/site-packages/librosa/core/constantq.py:1059: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.complex,


In [2]:
DEVICE_ID = 0

DATASET_LIST = [
#    ('handbag', '../../data/handbag_128.hdf5', 64),
#    ('handbag', '../../data/handbag_128.hdf5', 128),
#    ('shoes', '../../data/shoes_128.hdf5', 64),
#    ('shoes', '../../data/shoes_128.hdf5', 128),
    ('celeba_female', '/home/jovyan/data/CelebA_HQ_face_gender_dataset', 64),
    ('celeba_male', '/home/jovyan/data/CelebA_HQ_face_gender_dataset', 64),
#    ('aligned_anime_faces', '../../data/aligned_anime_faces', 64),
#    ('aligned_anime_faces', '../../data/aligned_anime_faces', 128),
]

assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_ID}')

In [3]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder('/home/jovyan/data/CelebA_HQ_face_gender_dataset')

In [4]:
dataset

Dataset ImageFolder
    Number of datapoints: 30000
    Root location: /home/jovyan/data/CelebA_HQ_face_gender_dataset

In [5]:
sampler, test_sampler = load_dataset('celeba_female', '/home/jovyan/data/CelebA_HQ_face_gender_dataset', img_size=64)

In [6]:
from tqdm import tqdm

In [7]:
for batch in tqdm(sampler.loader):
    pass

100%|██████████| 267/267 [01:43<00:00,  2.59it/s]


In [8]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              96
On-line CPU(s) list: 0-95
Thread(s) per core:  2
Core(s) per socket:  24
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz
Stepping:            4
CPU MHz:             3307.454
CPU max MHz:         3700.0000
CPU min MHz:         1200.0000
BogoMIPS:            5400.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-23,48-71
NUMA node1 CPU(s):   24-47,72-95
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmu

In [6]:
sampler.sample(1)

tensor([[[[-0.5686, -0.5686, -0.5686,  ..., -0.6784, -0.6784, -0.6784],
          [-0.5686, -0.5686, -0.5686,  ..., -0.6784, -0.6784, -0.6784],
          [-0.5686, -0.5686, -0.5686,  ..., -0.6784, -0.6784, -0.6784],
          ...,
          [-0.0039, -0.0039, -0.0039,  ..., -0.4118, -0.3961, -0.3882],
          [-0.0039, -0.0039, -0.0039,  ..., -0.3882, -0.3882, -0.3804],
          [-0.0039, -0.0039, -0.0039,  ..., -0.3725, -0.3725, -0.3725]],

         [[-0.5765, -0.5765, -0.5765,  ..., -0.6863, -0.6863, -0.6863],
          [-0.5765, -0.5765, -0.5765,  ..., -0.6863, -0.6863, -0.6863],
          [-0.5765, -0.5765, -0.5765,  ..., -0.6863, -0.6863, -0.6863],
          ...,
          [ 0.0431,  0.0431,  0.0431,  ..., -0.3882, -0.3725, -0.3647],
          [ 0.0431,  0.0431,  0.0431,  ..., -0.3647, -0.3647, -0.3569],
          [ 0.0431,  0.0431,  0.0431,  ..., -0.3490, -0.3490, -0.3490]],

         [[-0.5294, -0.5294, -0.5294,  ..., -0.6471, -0.6471, -0.6471],
          [-0.5294, -0.5294, -

In [7]:
for DATASET, DATASET_PATH, IMG_SIZE in tqdm(DATASET_LIST):
    print('Processing {}'.format(DATASET))
    sampler, test_sampler = load_dataset(DATASET, DATASET_PATH, img_size=IMG_SIZE)
    print('Dataset {} loaded'.format(DATASET))

    mu, sigma = get_loader_stats(test_sampler.loader)
    print('Trace of sigma: {}'.format(np.trace(sigma)))
    stats = {'mu' : mu.tolist(), 'sigma' : sigma.tolist()}
    print('Stats computed')

    filename = '{}_{}_test.json'.format(DATASET, IMG_SIZE)
    with open(filename, 'w') as fp:
        json.dump(stats, fp)
    print('States saved to {}'.format(filename))

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/2 [00:00<?, ?it/s]

Processing celeba_female
Dataset celeba_female loaded
Trace of sigma: 97.47934181453752
Stats computed
States saved to celeba_female_64_test.json
Processing celeba_male
Dataset celeba_male loaded
Trace of sigma: 114.14165870284711
Stats computed
States saved to celeba_male_64_test.json
